<a href="https://colab.research.google.com/github/uguazelli/langchain/blob/main/LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CodLab offers AI automation to clinics.

This virtual assistant will:

Greet the patient

Collect symptoms

Triage urgency (via LLM)

Offer appointment options if non-urgent

Send urgent cases to a human

Generate a summary for admin or doctor


In [ ]:
'''
        ┌──────────────┐
        │   START      │
        └─────┬────────┘
              ↓
     ┌─────────────────────┐
     │ Collect Symptoms     │◄─── Loop if info is unclear
     └────────┬────────────┘
              ↓
     ┌─────────────────────┐
     │   Triage w/ LLM      │
     └─────┬───────┬────────┘
           ↓       ↓
   ┌──────────┐   ┌─────────────────────┐
   │ Alert MD │   │ Book Appointment     │
   └──────────┘   └─────┬───────────────┘
                        ↓
              ┌─────────────────────┐
              │ Generate Summary    │
              └─────────┬───────────┘
                        ↓
                   ┌─────────┐
                   │  DONE   │
                   └─────────┘
'''

In [ ]:
!pip install -U -q langchain langchain-openai langgraph

In [10]:
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from google.colab import userdata
from typing import TypedDict, Optional
from typing import TypedDict, List, Optional

In [8]:
openai_key = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=openai_key
)


In [11]:
# Define state type
class GraphState(TypedDict):
    symptoms: Optional[str]
    triage_result: Optional[str]
    booking_confirmed: Optional[bool]


In [12]:
# Define LangGraph nodes (functions)
def collect_symptoms(state: GraphState):
    print("Collecting symptoms...")
    state["symptoms"] = "I have chest pain and shortness of breath"
    return state

def triage(state: GraphState):
    symptoms = state["symptoms"]
    print(f"Triage for: {symptoms}")
    response = llm.invoke(f"Classify the following symptoms as urgent or non-urgent: {symptoms}. Respond only with 'urgent' or 'non-urgent'.")
    state["triage_result"] = response.content.strip().lower()
    return state

def alert_doctor(state: GraphState):
    print("🚨 Urgent case! Alerting doctor...")
    return state

def book_appointment(state: GraphState):
    print("Booking appointment for next available slot...")
    state["booking_confirmed"] = True
    return state

In [13]:
# Build the graph
graph = StateGraph(GraphState)

graph.add_node("collect_symptoms", collect_symptoms)
graph.add_node("triage", triage)
graph.add_node("alert_doctor", alert_doctor)
graph.add_node("book_appointment", book_appointment)

graph.set_entry_point("collect_symptoms")
graph.add_edge("collect_symptoms", "triage")

def triage_router(state: GraphState):
    if state["triage_result"] == "urgent":
        return "alert_doctor"
    return "book_appointment"

graph.add_conditional_edges("triage", triage_router)
graph.set_finish_point("alert_doctor")
graph.set_finish_point("book_appointment")

app = graph.compile()


In [14]:
# Run the LangGraph app
final_state = app.invoke({})
print("\n✅ Final state:", final_state)

Triage for: I have chest pain and shortness of breath
🚨 Urgent case! Alerting doctor...

✅ Final state: {'symptoms': 'I have chest pain and shortness of breath', 'triage_result': 'urgent'}
